# <font color = 'orange'> Preparing Base Model

---

In [1]:
import os

In [2]:
%pwd

'e:\\PW Skills - Data Science\\Projects\\09_Chest_Cancer_Classification\\notebook'

In [3]:
os.chdir('../')

In [4]:
%pwd

'e:\\PW Skills - Data Science\\Projects\\09_Chest_Cancer_Classification'

---

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int

In [6]:
from chest_cancer_classifier.constants import *
from chest_cancer_classifier.utils.common import read_yaml, create_directories


class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES
        )

        return prepare_base_model_config

In [7]:
from pathlib import Path
import tensorflow as tf
from keras import Sequential
from keras.applications.vgg16 import VGG16
from keras.layers import Dense,Flatten
from keras.optimizers import Adam
from keras.losses import BinaryCrossentropy


class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config


    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    
    def get_base_model(self):
        self.model = VGG16(
            weights = self.config.params_weights,
            input_shape = self.config.params_image_size,
            include_top = self.config.params_include_top
        )
        self.save_model(path = self.config.base_model_path, model = self.model)

    
    @staticmethod
    def __create_custom_model(base_model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            base_model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in base_model.layers[:-freeze_till]:
                layer.trainable = False

        custom_model = Sequential()
        
        custom_model.add(base_model)
        custom_model.add(Flatten())
        custom_model.add(Dense(256, activation='relu'))
        custom_model.add(Dense(1, activation='sigmoid'))

        custom_model.compile(
            optimizer = Adam(learning_rate = learning_rate),
            loss = BinaryCrossentropy(),
            metrics = ["accuracy"]
        )

        custom_model.summary()
        return custom_model
    

    def create_custom_model(self):
        self.custom_model = self.__create_custom_model(
            base_model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )
        self.save_model(path=self.config.updated_base_model_path, model=self.custom_model)

In [8]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config = prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.create_custom_model()
except Exception as e:
    raise e

[2024-04-23 10:12:34,789: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-04-23 10:12:34,799: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-23 10:12:34,803: INFO: common: created directory at: artifacts]
[2024-04-23 10:12:34,803: INFO: common: created directory at: artifacts/prepare_base_model]


[2024-04-23 10:12:36,008: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 256)               6422784   
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 21,137,729
Trainable params: 6,423,041
Non-trainable params: 14,714,688
_________________________________

---